__IMPORTING PYTHON MODULES__

In [3]:
import pandas as pd
import numpy as np
import os
import io
import requests
from bs4 import BeautifulSoup as bs
import re
import string


__READING INPUT EXCEL SHEET__

In [2]:
input_data = pd.read_excel("input.xlsx")


__SCRAPING CODE (DATA ALREADY SCRAPED AND SAVED IN DIFFERENT FOLDER)__

In [ ]:
# for i in range(len(input_data)):
#     if(i in [7,20,107]):
#         continue
#     else:
#         url = input_data['URL'][i]
#         page = requests.get(url)
#         soup = bs(page.content,'html.parser')
#         heading = soup.find('h1', class_ = ["entry-title","tdb-title-text"]).text
#         para = soup.find_all('p', attrs={"class": None})
#         with open(os.path.join('datatxt','{}.txt'.format(input_data['URL_ID'][i])), 'w', encoding = 'utf8') as f:
#             f.write(heading)
#             f.write('\n')
#         for p in para:
#             with open(os.path.join('datatxt','{}.txt'.format(input_data['URL_ID'][i])), 'a', encoding = 'utf8') as f:
#                 f.write(p.text)
#                 f.write('\n')

__IMPORTING NLTK MODULES

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import cmudict
nltk.download('cmudict')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading cmudict: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading averaged_perceptron_tagger: <

False

In [5]:
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


__DEFINING FUNCTIONS FOR SYLLABLE COUNT__

In [ ]:
d = cmudict.dict()

def nsyl(word):
    count = 0
    for x in d[word.lower()][0]:
        if(x[-1].isdigit()):
            count+=1
    return count

def syllables_count(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

__FORMING OUTPUT DATA STRUCTURE__

In [ ]:
output_data = pd.read_excel('Output Data Structure.xlsx')


for idx in range(len(output_data)):
    if(idx in [7,20,107]):
        continue
   
    else:
        with open(os.path.join('datatxt','{}.txt'.format(output_data['URL_ID'][idx])),'r', encoding = 'utf8') as file:
            data = file.read()

        Tokenizer = TreebankWordTokenizer()
        tokenized_word = Tokenizer.tokenize(data)
        
        para = [p for p in data.split('\n') if p]
        x=[]
        for p in para:
            x.append(sent_tokenize(p))
        tokenized_sent = [item for sublist in x for item in sublist]

        english_stops = set(stopwords.words('english'))
        words = [word for word in tokenized_word if word not in english_stops]

        lm = WordNetLemmatizer()
        punct = [i for i in string.punctuation]
        final = [lm.lemmatize(word) for word in words if word not in punct]

        with open('positive-words.txt','r') as f1:
            list = f1.read()
            positive_words = list.split('\n')
        with open('negative-words.txt','r') as f2:
            list = f2.read()
            negative_words = list.split('\n')

        syllable = []
        for word in final:
            if(word in d):
                syllable.append(nsyl(word))
            else:
                syllable.append(syllables_count(word))

        complex_words = [x for x in syllable if x>2]

        char_word = [len(word) for word in final]  

        pos = nltk.pos_tag(word_tokenize(data))
        per_pronoun = [pos[i][0] for i in range(len(pos)) if pos[i][1] == 'PRP']

        positive_score = len([word for word in final if word in positive_words])
        negative_score = len([word for word in final if word in negative_words])
        polarity_score = (positive_score-negative_score)/((positive_score + negative_score) + 0.000001)
        subjectivity_score = (positive_score + negative_score)/ (len(final) + 0.000001)
        avg_sent_len = len(final)/len(tokenized_sent)
        complex_prcnt = round(100*(len(complex_words)/len(final)),5)
        fog_index = 0.4*(avg_sent_len + complex_prcnt)
        syl_per_wrd = sum(syllable)/len(final)
        per_pro_count = len(per_pronoun)
        avg_wrd_len = sum(char_word)/len(final)


        output_data.iloc[idx,2:15] = [positive_score, negative_score, polarity_score, subjectivity_score,
                                   avg_sent_len, complex_prcnt, fog_index, avg_sent_len, len(complex_words),
                                   len(final), syl_per_wrd, per_pro_count, avg_wrd_len]





__EDITING AND SAVING FILE IN XLSX FORMAT__

In [ ]:
output_data

In [ ]:
d